In [22]:
import numpy as np
import mesh

# EXEMPLOS BIBLIOTECA EDSON

In [23]:
nome = '../../malhasMSH/circ2_tst_Hua_v2_2_lc_especial.msh'
#nome = '../../malhasMSH/circ8_anom4_tst_Hua_v2_2_lc_especial.msh'

MinhaMalha = mesh.HuaElectrodes2DMeshEdson(2, nome_msh=nome, altura2D = 0.02)
MinhaMalha.ReadMesh() 

Reading ../../malhasMSH/circ2_tst_Hua_v2_2_lc_especial.msh.

msh_physical_groups found (type triangle): [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
 1000 1000 1000 1000].
Physical tags found (type triangle): [1000].
Physical tags: lines: [5001 5002]; points: [10000]
2 electrodes found.
MSH file with 18 elements and 17 nodes.
ElectrodeNodes: [17 18]
26 Elements and 19 Nodes found on model.
GndNode: 10
4 11 16
KGeo1 [[ 0.03498348  0.00035101 -0.03533448]
 [ 0.00035101  0.00286201 -0.00546489]
 [-0.03533448 -0.00546489  0.0385475 ]]
9 13 15
KGeo1 [[ 0.03498348  0.00035101 -0.03533448]
 [ 0.00035101  0.00286201 -0.00546489]
 [-0.03533448 -0.00546489  0.0385475 ]]
14 0 16
KGeo1 [[ 0.00286201  0.00035101 -0.00321302]
 [ 0.00035101  0.03498348 -0.00569177]
 [-0.00321302 -0.00569177  0.0385475 ]]
12 5 15
KGeo1 [[ 0.00286201  0.00035101 -0.00321302]
 [ 0.00035101  0.03498348 -0.00569177]
 [-0.00321302 -0.00569177  0.0385475 ]]
8 9 15
KGeo1 [[ 0.08029881 -0.07819706 -0

In [24]:
print(MinhaMalha.KGlobal)

None


In [25]:
print(MinhaMalha.Elements[2])
print(f"Centroid: {MinhaMalha.Elements[2].Centroid}")
print(f"KGeo: \n{MinhaMalha.Elements[2].KGeo}")

Centroid: [ 0.11114379 -0.04717695  0.        ]
KGeo: 
[[ 0.00286201  0.00035101 -0.00321302]
 [ 0.00035101  0.03498348 -0.00569177]
 [-0.00321302 -0.00569177  0.0385475 ]]


In [26]:
meus_sigmas = {}
meus_sigmas[1000] = 0.1
meus_sigmas[5001] = 0.2
meus_sigmas[5002] = 0.2
MinhaMalha.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)

MinhaMalha.CalcKGlobal() # calculando KGlobal usando Sigmas

MinhaMalha.KGlobal




KGlobal =  MinhaMalha.KGlobal

print('KGlobal', KGlobal[17])

CHAVES recebidas em dic: [1000, 5001, 5002]
KGlobal [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [27]:
''''''
meus_sigmas = {}
meus_sigmas[1000] = 0.1
#meus_sigmas[2] = 0.2
MinhaMalha.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)
''''''

CHAVES recebidas em dic: [1000]


KeyError: np.int32(5001)

In [ ]:
#MinhaMalha.CalcKGlobal() # calculando KGlobal usando Sigmas

In [ ]:
#MinhaMalha.KGlobal

# EXEMPLOS BIBLIOTECAS OLD

In [ ]:
'''
nome = '../../malhasMSH/circ8_anomalia5.msh'

MinhaMalha = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
MinhaMalha.ReadMesh()
'''

"\nnome = '../../malhasMSH/circ8_anomalia5.msh'\n\nMinhaMalha = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nMinhaMalha.ReadMesh()\n"

In [ ]:
'''
print(MinhaMalha.Elements[2])
print(f"Centroid: {MinhaMalha.Elements[2].Centroid}")
print(f"KGeo: \n{MinhaMalha.Elements[2].KGeo}")
'''

'\nprint(MinhaMalha.Elements[2])\nprint(f"Centroid: {MinhaMalha.Elements[2].Centroid}")\nprint(f"KGeo: \n{MinhaMalha.Elements[2].KGeo}")\n'

In [ ]:
'''
meus_sigmas = {}
meus_sigmas[1] = 0.1
meus_sigmas[2] = 0.2
MinhaMalha.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)
'''

'\nmeus_sigmas = {}\nmeus_sigmas[1] = 0.1\nmeus_sigmas[2] = 0.2\nMinhaMalha.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)\n'

In [ ]:
#MinhaMalha.CalcKGlobal() # calculando KGlobal usando Sigmas

In [ ]:
#MinhaMalha.KGlobal

In [ ]:
'''
M2 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
M2.ReadMesh()
meus_rhos = {}
meus_rhos[1] = 10
meus_rhos[2] = 5
M2.SetRhoPhysicaEntity(meus_rhos) # Informando rho (e já calculando o sigma de cada elemento)
M2.CalcKGlobal() # calculando KGlobal usando Sigmas
M2.KGlobal
'''

'\nM2 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nM2.ReadMesh()\nmeus_rhos = {}\nmeus_rhos[1] = 10\nmeus_rhos[2] = 5\nM2.SetRhoPhysicaEntity(meus_rhos) # Informando rho (e já calculando o sigma de cada elemento)\nM2.CalcKGlobal() # calculando KGlobal usando Sigmas\nM2.KGlobal\n'

In [ ]:
'''
M3 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
M3.ReadMesh()
meus_rhos = {}
meus_rhos[1] = 10
meus_rhos[2] = 5
M3.SetRhoPhysicaEntity(meus_rhos) # Informando rho (e já calculando o sigma de cada elemento)
M3.FlagRhoBased = True
M3.CalcKGlobal() # calculando KGlobal usando Rho
M3.KGlobal
'''

'\nM3 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nM3.ReadMesh()\nmeus_rhos = {}\nmeus_rhos[1] = 10\nmeus_rhos[2] = 5\nM3.SetRhoPhysicaEntity(meus_rhos) # Informando rho (e já calculando o sigma de cada elemento)\nM3.FlagRhoBased = True\nM3.CalcKGlobal() # calculando KGlobal usando Rho\nM3.KGlobal\n'

In [ ]:
'''
M4 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
M4.ReadMesh()
meus_sigmas = {}
meus_sigmas[1] = 0.1
meus_sigmas[2] = 0.2
M4.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)
M4.FlagRhoBased = True
M4.CalcKGlobal() # calculando KGlobal usando Rho
M4.KGlobal
'''

'\nM4 = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nM4.ReadMesh()\nmeus_sigmas = {}\nmeus_sigmas[1] = 0.1\nmeus_sigmas[2] = 0.2\nM4.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)\nM4.FlagRhoBased = True\nM4.CalcKGlobal() # calculando KGlobal usando Rho\nM4.KGlobal\n'

In [ ]:
# M4

Comparação ajustando por valores, physical grous ou vetores

In [ ]:
'''
Mpg = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
Mpg.ReadMesh()
meus_sigmas = {}
meus_sigmas[1] = 0.1
meus_sigmas[2] = 0.1
Mpg.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)
Mpg.FlagRhoBased = True
Mpg.CalcKGlobal() # calculando KGlobal usando Rho
Mpg.KGlobal
'''

'\nMpg = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nMpg.ReadMesh()\nmeus_sigmas = {}\nmeus_sigmas[1] = 0.1\nmeus_sigmas[2] = 0.1\nMpg.SetSigmaPhysicaEntity(meus_sigmas) # Informando sigma (e já calculando o rho de cada elemento)\nMpg.FlagRhoBased = True\nMpg.CalcKGlobal() # calculando KGlobal usando Rho\nMpg.KGlobal\n'

In [ ]:
'''
Mval = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
Mval.ReadMesh()
value = 0.1
Mval.SetSigmaValue(value) # Informando sigma (e já calculando o rho de cada elemento)
Mval.FlagRhoBased = True
Mval.CalcKGlobal() # calculando KGlobal usando Rho
Mval.KGlobal
'''

'\nMval = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nMval.ReadMesh()\nvalue = 0.1\nMval.SetSigmaValue(value) # Informando sigma (e já calculando o rho de cada elemento)\nMval.FlagRhoBased = True\nMval.CalcKGlobal() # calculando KGlobal usando Rho\nMval.KGlobal\n'

In [ ]:
'''
Mvec = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)
Mvec.ReadMesh()
vec = np.ones(Mvec.NumberOfElements) * 0.1
Mvec.SetSigmaElements(vec) # Informando sigma (e já calculando o rho de cada elemento)
Mvec.FlagRhoBased = True
Mvec.CalcKGlobal() # calculando KGlobal usando Rho
Mvec.KGlobal
'''

'\nMvec = mesh.PointElectrodes2DMeshEdson(8, nome_msh=nome, altura2D = 0.02)\nMvec.ReadMesh()\nvec = np.ones(Mvec.NumberOfElements) * 0.1\nMvec.SetSigmaElements(vec) # Informando sigma (e já calculando o rho de cada elemento)\nMvec.FlagRhoBased = True\nMvec.CalcKGlobal() # calculando KGlobal usando Rho\nMvec.KGlobal\n'